In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import seal
from seal import *
import time
import pickle

In [2]:
#Evaluator initializes an encryption scheme

parms = EncryptionParameters(scheme_type.ckks)

#Evaluator loads client's parameters

parms.load('encr_parms')

#Evaluator initializes context with client's parms

context = SEALContext(parms)

ckks_encoder = CKKSEncoder(context)
slot_count = ckks_encoder.slot_count()

keygen = KeyGenerator(context)
public_key = keygen.create_public_key()
public_key.load(context, 'pubkey')


galois_keys = keygen.create_galois_keys()
galois_keys.load(context, 'galkeys')


relin_keys = keygen.create_relin_keys()
relin_keys.load(context, 'relinkeys')

scale = pickle.load(open('scale','rb'))

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)

In [3]:
#This must be set to 198

num_test_samples = 300

In [4]:
start = time.time()

ct_data = []

for i in range(num_test_samples):
    ct_sample = []
    for j in range(12):
        pt_init = ckks_encoder.encode(0.,scale)
        ct_init = encryptor.encrypt(pt_init)
        ct_init.load(context, 'data_ct_[%i,%i]' % (i,j))
        ct_sample.append(ct_init)
    ct_data.append(ct_sample)
        
end = time.time()
print(f"Time to load encrypted data: {(end-start):.3f}s")

Time to load encrypted data: 26.942s


In [5]:
model_index_data = pickle.load(open('model_indices','rb'))

In [6]:
start = time.time()

ct_models = []

for i in range(5):
    ct_sample = []
    for j in range(len(model_index_data[i][0])):
        pt_init = ckks_encoder.encode(0.,scale)
        ct_init = encryptor.encrypt(pt_init)
        ct_init.load(context, 'model_ct[%i][%i]' % (i,j))
        ct_sample.append(ct_init)
    ct_models.append(ct_sample)
        
end = time.time()
print(f"Time to load encrypted model parameters: {(end-start):.3f}s")

Time to load encrypted model parameters: 2.517s


In [7]:
#Evaluate all models

start = time.time()

result = []
for m in range(num_test_samples):


    temp = []
    for i in range(len(ct_models)):
        for j in range(len(ct_models[i])):
        
            temp.append(evaluator.multiply(ct_models[i][j],ct_data[m][model_index_data[i][0][j]]))
            evaluator.relinearize_inplace(temp[len(temp)-1],relin_keys)
            evaluator.rescale_to_next_inplace(temp[len(temp)-1])
            evaluator.rotate_vector_inplace(temp[len(temp)-1],model_index_data[i][1][j]-i,galois_keys)
    
    
    result.append(evaluator.add_many(temp))
    
end = time.time()
print(f"Time to evaluate all models on all test samples: {(end-start):.3f}s")

Time to evaluate all models on all test samples: 319.502s


In [8]:
#Save result to send back to client for decryption

for m in range(len(result)):
    result[m].save('result[%i]'%m)